In [1]:
import radiomics
import os
import SimpleITK as sitk
import pandas as pd
from radiomics import featureextractor
import numpy as np
import matplotlib.pyplot as plt
import ITHscore
import scipy


In [ ]:
dicom_path = r'C:\ISPY-2_700\manifest-1698645890562\ISPY2'
ID_total = []
result_result = []

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [3, 5]
settings['normalize'] = True
# settings['resampledPixelSpacing'] = [1,1,1]
# settings['resampledPixelSpacing'] = [2,2,2]
# settings['resampledPixelSpacing'] = [3,3,3]
# settings['resampledPixelSpacing'] = [4,4,4]
settings['resampledPixelSpacing'] = [5,5,5]

settings['interpolator'] = sitk.sitkBSpline

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableAllImageTypes()
# extractor.enableImageTypeByName('Original')
# extractor.enableImageTypeByName('Wavelet')
# extractor.enableImageTypeByName('LoG')
extractor.enableAllFeatures()

for item_0 in os.listdir(dicom_path):
    ID = item_0[-6:]
    item_1 = os.path.join(dicom_path, item_0)
    for item_2 in os.listdir(item_1):
        item_3 = os.path.join(item_1, item_2)
        for item_4 in os.listdir(item_3):
            item_5 = os.path.join(item_3,item_4)
            if "Mask" in item_5:
                for item_6 in os.listdir(item_5):
                    mask_path = os.path.join(item_5,item_6)
                    mask_image = sitk.ReadImage(mask_path)
                    mask_array = sitk.GetArrayFromImage(mask_image)
                    mask_array_max = np.max(mask_array)+1
                    mask_array[mask_array == 0] = mask_array_max
                    mask_array[mask_array < mask_array_max] = 0
                    mask_array[mask_array != 0] = 1
            elif "DCE" in item_5 and "gz" not in item_5:
                # for item_6 in os.listdir(item_5):
                    reader = sitk.ImageSeriesReader()
                    dicom = reader.GetGDCMSeriesFileNames(item_5)
                    reader.SetFileNames(dicom)
                    img = reader.Execute()
                    compressed_filename = item_5 + ".nii.gz"
                    sitk.WriteImage(img, compressed_filename, True)
                    dicom_image = sitk.ReadImage(compressed_filename)
                    image_array = sitk.GetArrayFromImage(dicom_image)

        try:
            print(ID)
            k = image_array.shape[0]/mask_array.shape[0]
            image_array = image_array[mask_array.shape[0]*2:mask_array.shape[0]*3,:,:]
            dicom_image = sitk.GetImageFromArray(image_array)
            mask_image = sitk.GetImageFromArray(mask_array)

            #label代表提取特征区域的像素值，可以不设置，则代表提取全部，不仅是ROI
            
            result = extractor.execute(dicom_image, mask_image,label = 1)

            result_result.append(result)
            ID_total.append(ID)
        # 1/0
        
        except Exception as e:
            print(ID)
result = pd.DataFrame(result_result)
csv_filename_1 = "pyra_feature_normal_resample[555].csv"
result.to_csv(csv_filename_1, index=False)


In [ ]:
dicom_path = r'C:\ISPY-2_700\manifest-1698645890562\ISPY2'
ID_total = []
result_result = []

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [3, 5]
settings['normalize'] = True
# settings['normalize'] = None
# settings['resampledPixelSpacing'] = [1,1,1]
settings['resampledPixelSpacing'] = [2,2,2]
# settings['resampledPixelSpacing'] = [3,3,3]
# settings['resampledPixelSpacing'] = None

settings['interpolator'] = sitk.sitkBSpline

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableAllImageTypes()
# extractor.enableImageTypeByName('Original')
# extractor.enableImageTypeByName('Wavelet')
# extractor.enableImageTypeByName('LoG')
extractor.enableAllFeatures()

for item_0 in os.listdir(dicom_path):
    ID = item_0[-6:]
    item_1 = os.path.join(dicom_path, item_0)
    for item_2 in os.listdir(item_1):
        item_3 = os.path.join(item_1, item_2)
        for item_4 in os.listdir(item_3):
            item_5 = os.path.join(item_3,item_4)
            if "Mask" in item_5:
                for item_6 in os.listdir(item_5):
                    mask_path = os.path.join(item_5,item_6)
                    mask_image = sitk.ReadImage(mask_path)
                    mask_array = sitk.GetArrayFromImage(mask_image)
                    
            elif "DCE" in item_5 and "gz" not in item_5:
                # for item_6 in os.listdir(item_5):
                    reader = sitk.ImageSeriesReader()
                    dicom = reader.GetGDCMSeriesFileNames(item_5)
                    reader.SetFileNames(dicom)
                    img = reader.Execute()
                    compressed_filename = item_5 + ".nii.gz"
                    sitk.WriteImage(img, compressed_filename, True)
                    dicom_image = sitk.ReadImage(compressed_filename)
                    image_array = sitk.GetArrayFromImage(dicom_image)

        try:
            print(ID)
            k = image_array.shape[0]/mask_array.shape[0]
            image_array = image_array[mask_array.shape[0]*2:mask_array.shape[0]*3,:,:]
            dicom_image = sitk.GetImageFromArray(image_array)
            mask_image = sitk.GetImageFromArray(mask_array)
            # dicom_image = resample_img(dicom_image)
            # dicom_image = sitk.Normalize(dicom_image)
            # mask_image = resample_img(mask_image,is_label=True)
    # #
            image_array = sitk.GetArrayFromImage(dicom_image)
            mask_array = sitk.GetArrayFromImage(mask_image)

    #         coords = np.argwhere(mask_array < 30)
    #         min_coords = np.min(coords, axis=0)
    #         max_coords = np.max(coords, axis=0)
    #         cropped_image_array = image_array[min_coords[0]:max_coords[0]+1, min_coords[1]:max_coords[1]+1, min_coords[2]:max_coords[2]+1]
    #         cropped_mask_array = mask_array[min_coords[0]:max_coords[0]+1, min_coords[1]:max_coords[1]+1, min_coords[2]:max_coords[2]+1]

            cropped_mask_array_max = np.max(mask_array)+1
            mask_array[mask_array == 0] = cropped_mask_array_max
            mask_array[mask_array < cropped_mask_array_max] = 0
            mask_array[mask_array != 0] = 1

            dicom_image = sitk.GetImageFromArray(image_array)
            mask_image = sitk.GetImageFromArray(mask_array)

            #label代表提取特征区域的像素值，可以不设置，则代表提取全部，不仅是ROI
            
            result = extractor.execute(dicom_image, mask_image,label = 1)

            result_result.append(result)
            ID_total.append(ID)
        
        
        except Exception as e:
            print(ID)
        


In [ ]:
result = pd.DataFrame(result_result)
csv_filename_1 = "feature.csv"
result.to_csv(csv_filename_1, index=False)


In [ ]:
dicom_path = r'C:\ISPY-2_700\manifest-1698645890562\ISPY2'

for item_0 in os.listdir(dicom_path):
    ID = item_0[-6:]
    item_1 = os.path.join(dicom_path, item_0)
    for item_2 in os.listdir(item_1):
        item_3 = os.path.join(item_1, item_2)
        for item_4 in os.listdir(item_3):
            item_5 = os.path.join(item_3,item_4)
            if "Mask" in item_5:
                for item_6 in os.listdir(item_5):
                    mask_path = os.path.join(item_5,item_6)
                    mask_image = sitk.ReadImage(mask_path)
                    mask_array = sitk.GetArrayFromImage(mask_image)
            elif "DCE" in item_5 and "gz" not in item_5:
                # for item_6 in os.listdir(item_5):
                    reader = sitk.ImageSeriesReader()
                    dicom = reader.GetGDCMSeriesFileNames(item_5)
                    sitk.ReadImage(dicom)
                    reader.SetFileNames(dicom)
                    img = reader.Execute()
                    compressed_filename = item_5 + ".nii.gz" 
                    sitk.WriteImage(img, compressed_filename, True)
                    dicom_image = sitk.ReadImage(compressed_filename)
                    image_array = sitk.GetArrayFromImage(dicom_image)
        try:
            print(ID)
            k = image_array.shape[0]/mask_array.shape[0]
            image_array = image_array[mask_array.shape[0]*2:mask_array.shape[0]*3,:,:]
            
            dicom_image = sitk.GetImageFromArray(image_array)
            mask_image = sitk.GetImageFromArray(mask_array)
            dicom_image = resample_img(dicom_image)
            mask_image = resample_img(mask_image,is_label=True)
            mask_array = sitk.GetArrayFromImage(mask_image)

            coords = np.argwhere(mask_array < 30)
            min_coords = np.min(coords, axis=0)
            max_coords = np.max(coords, axis=0)
            dicom_image = sitk.GetArrayFromImage(sitk.Normalize(dicom_image))
            mask_array = sitk.GetArrayFromImage(mask_image)

            cropped_image_array = dicom_image[min_coords[0]:max_coords[0]+1, min_coords[1]:max_coords[1]+1, min_coords[2]:max_coords[2]+1]
            cropped_mask_array = mask_array[min_coords[0]:max_coords[0]+1, min_coords[1]:max_coords[1]+1, min_coords[2]:max_coords[2]+1]

            cropped_image_image = sitk.GetImageFromArray(cropped_image_array)
            cropped_mask_iamge = sitk.GetImageFromArray(cropped_mask_array)
            dicom_filename = ID + "resample_ori.nii.gz" 
            mask_filename = ID + "resample_mask.nii.gz" 


            sitk.WriteImage(cropped_image_image,dicom_filename,True)
            sitk.WriteImage(cropped_mask_iamge,mask_filename,True)
        
            #label代表提取特征区域的像素值，可以不设置，则代表提取全部，不仅是ROI

            # result = extractor.execute(dicom_image, mask_image,label = 1)

            # result_result.append(result)
            # ID_total.append(ID)


        except Exception as e:
            print(ID)


In [5]:
image_path = r"C:\Users\JackonHong\Desktop\1.resample[555]2.normal3.slice\image"
mask_path = r"C:\Users\JackonHong\Desktop\1.resample[555]2.normal3.slice\mask"
image_list = os.listdir(image_path)
image_list.sort()
mask_list = os.listdir(mask_path)
mask_list.sort()

result_result = []

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [3, 5]
settings['normalize'] = None
settings['resampledPixelSpacing'] = None

settings['interpolator'] = sitk.sitkBSpline

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableAllImageTypes()
extractor.enableAllFeatures()
try:
    for i in range(len(mask_list)):
        ID = image_list[i][0:6]
        image = sitk.ReadImage(image_path + "\\" + image_list[i])
        mask = sitk.ReadImage(mask_path + "\\" + mask_list[i])
        image_array = sitk.GetArrayFromImage(image)
        mask_array = sitk.GetArrayFromImage(mask)
        cropped_mask_array_max = np.max(mask_array)+1
        mask_array[mask_array == 0] = cropped_mask_array_max
        mask_array[mask_array < cropped_mask_array_max] = 0
        mask_array[mask_array != 0] = 1
        iteration = 2 # newSpacing是1mm,扩展3mm
        mask_img_arr_expand = expand_dilation(mask_array,iterations=iteration)
        mask_img_arr_border = diffROI(mask_array,mask_img_arr_expand)
        mask_img_arr_border = sitk.GetImageFromArray(mask_img_arr_border)
        img_border = sitk.GetImageFromArray(image_array)
        result = extractor.execute(img_border, mask_img_arr_border, label = 1)
        result_result.append(result)

except Exception as e:
    print(ID)

result = pd.DataFrame(result_result)
csv_filename_1 = "pyra_feature_[555]_border2.csv"
result.to_csv(csv_filename_1, index=False)


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Calculating Local Binary Pattern in 2D, but extracting features in 3D. Use with caution!
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Could not load required package "scipy" or "trimesh", cannot implement filter LBP 3D
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is

999733


In [ ]:
image_path = r"C:\Users\JackonHong\Desktop\1.resample[333]2.normal3.slice\image"
mask_path = r"C:\Users\JackonHong\Desktop\1.resample[333]2.normal3.slice\mask"
image_list = os.listdir(image_path)
image_list.sort()
mask_list = os.listdir(mask_path)
mask_list.sort()

result_result = []

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [3, 5]
settings['normalize'] = None
settings['resampledPixelSpacing'] = None

settings['interpolator'] = sitk.sitkBSpline

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableAllImageTypes()
extractor.enableAllFeatures()
try:
    for i in range(len(mask_list)):
        ID = image_list[i][0:6]
        image = sitk.ReadImage(image_path + "\\" + image_list[i])
        mask = sitk.ReadImage(mask_path + "\\" + mask_list[i])
        image_array = sitk.GetArrayFromImage(image)
        mask_array = sitk.GetArrayFromImage(mask)
        cropped_mask_array_max = np.max(mask_array)+1
        mask_array[mask_array == 0] = cropped_mask_array_max
        mask_array[mask_array < cropped_mask_array_max] = 0
        mask_array[mask_array != 0] = 1
        iteration = 3 # newSpacing是1mm,扩展3mm
        mask_img_arr_expand = expand_dilation(mask_array,iterations=iteration)
        mask_img_arr_border = diffROI(mask_array,mask_img_arr_expand)
        mask_img_arr_border = sitk.GetImageFromArray(mask_img_arr_border)
        img_border = sitk.GetImageFromArray(image_array)

        result = extractor.execute(img_border, mask_img_arr_border, label = 1)
        result_result.append(result)

except Exception as e:
    print(ID)

result = pd.DataFrame(result_result)
csv_filename_1 = "pyra_feature_[333]_border3.csv"
result.to_csv(csv_filename_1, index=False)


In [ ]:
image_path = r"C:\Users\JackonHong\Desktop\1.resample[333]2.normal3.slice\image"
mask_path = r"C:\Users\JackonHong\Desktop\1.resample[333]2.normal3.slice\mask"
image_list = os.listdir(image_path)
image_list.sort()
mask_list = os.listdir(mask_path)
mask_list.sort()

result_result = []

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [3, 5]
settings['normalize'] = None
settings['resampledPixelSpacing'] = None

settings['interpolator'] = sitk.sitkBSpline

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableAllImageTypes()
extractor.enableAllFeatures()
try:
    for i in range(len(mask_list)):
        ID = image_list[i][0:6]
        image = sitk.ReadImage(image_path + "\\" + image_list[i])
        mask = sitk.ReadImage(mask_path + "\\" + mask_list[i])
        image_array = sitk.GetArrayFromImage(image)
        mask_array = sitk.GetArrayFromImage(mask)
        cropped_mask_array_max = np.max(mask_array)+1
        mask_array[mask_array == 0] = cropped_mask_array_max
        mask_array[mask_array < cropped_mask_array_max] = 0
        mask_array[mask_array != 0] = 1
        iteration = 4 # newSpacing是1mm,扩展3mm
        mask_img_arr_expand = expand_dilation(mask_array,iterations=iteration)
        mask_img_arr_border = diffROI(mask_array,mask_img_arr_expand)
        mask_img_arr_border = sitk.GetImageFromArray(mask_img_arr_border)
        img_border = sitk.GetImageFromArray(image_array)

        result = extractor.execute(img_border, mask_img_arr_border, label = 1)
        result_result.append(result)

except Exception as e:
    print(ID)

result = pd.DataFrame(result_result)
csv_filename_1 = "pyra_feature_[333]_border4.csv"
result.to_csv(csv_filename_1, index=False)


In [ ]:
def resample_img(itk_image, out_spacing=[5.0, 5.0, 5.0], is_label=False):
    # resample images to 2mm spacing with simple itk

    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

In [2]:
def expand_dilation(mask_img_arr,iterations = 5):
    shape_nrrd = mask_img_arr.shape
    mask_img_arr_expand = np.zeros(shape_nrrd)
    for index in range(shape_nrrd[0]):
        mask_img_arr_expand[index,:,:] = scipy.ndimage.binary_dilation(mask_img_arr[index,:,:],iterations = iterations).astype('uint16')
    return mask_img_arr_expand


def diffROI(mask_img_arr,mask_img_arr_expand):
# 通过传入原始ROI mask_img_arr和扩展了5mm的ROI mask_img_arr_expand
# 返回扩展的5mm的ROI区域
    # 首先把二值ROI都转为0和1 
    mask_img_arr[mask_img_arr != 0] = 1
    mask_img_arr_expand[mask_img_arr_expand != 0] = 1

    shape_nrrd = mask_img_arr.shape
    mask_img_arr_border = np.zeros(shape_nrrd)   
    for index in range(shape_nrrd[0]):
        for x in range(shape_nrrd[1]):
            for y in range(shape_nrrd[2]):
                if mask_img_arr[index,x,y] != mask_img_arr_expand[index,x,y]:
                    mask_img_arr_border[index,x,y] = 1
    return mask_img_arr_border
    

def array2nii(image_array, out_path, NIIimage_resample):
    ## image_array是矩阵，out_path是带文件名的路径，NIIimage_resample是sitk_obj
    # 1.构建nrrd阅读器
    image2 = NIIimage_resample
    # 2.将整合后的数据转为array，并获取dicom文件基本信息
    origin = image2.GetOrigin()  # x, y, z
    spacing = image2.GetSpacing()  # x, y, z
    direction = image2.GetDirection()  # x, y, z
    # 3.将array转为img，并保存为.nii.gz
    image3 = sitk.GetImageFromArray(image_array)
    image3.SetSpacing(spacing)
    image3.SetDirection(direction)
    image3.SetOrigin(origin)
    sitk.WriteImage(image3, out_path)
